In [11]:
import requests, re, json

# pick one row and extract first url
def extract_first_url(raw):
    if pd.isna(raw) or raw is None:
        return None
    s = str(raw)
    try:
        arr = json.loads(s.replace("'", '"'))
        if isinstance(arr, list) and arr: 
            return arr[0]
    except Exception:
        pass
    m = re.findall(r"https?://[^\s,'\"]+", s)
    return m[0] if m else None

test_url = extract_first_url(df["images"].iloc[0])
print("Test URL:", test_url)

r = requests.get(test_url, timeout=15)
print("Plain request status:", r.status_code)


Test URL: https://m.media-amazon.com/images/I/416WaLx10jL._SS522_.jpg 
Plain request status: 400


In [12]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from PIL import Image
from io import BytesIO

# Browser-like headers
BROWSER_HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/124.0 Safari/537.36"
    ),
    "Accept": "image/avif,image/webp,image/apng,image/*,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
    "Sec-Fetch-Site": "cross-site",
    "Sec-Fetch-Mode": "no-cors",
    "Sec-Fetch-Dest": "image",
    "Referer": "https://www.amazon.in/",  # helps many Amazon image URLs
    "Connection": "keep-alive",
}

# Session with retries
def make_session():
    s = requests.Session()
    retries = Retry(total=3, backoff_factor=0.5, status_forcelist=[400, 403, 408, 429, 500, 502, 503, 504])
    s.mount("https://", HTTPAdapter(max_retries=retries))
    s.mount("http://", HTTPAdapter(max_retries=retries))
    return s

session = make_session()

def fetch_image(url: str):
    if not url:
        return None, "no_url"
    try:
        # allow redirects, stream raw, browser-like headers
        r = session.get(url, headers=BROWSER_HEADERS, timeout=20, allow_redirects=True, stream=True)
        if r.status_code != 200:
            return None, f"status_{r.status_code}"
        # PIL can open from raw stream or bytes
        img = Image.open(BytesIO(r.content)).convert("RGB")
        return img, "ok"
    except Exception as e:
        return None, f"err_{type(e).__name__}"


In [14]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("clip-ViT-B-32")
CATEGORIES = ["chair", "sofa", "table", "bed", "shelf", "lamp", "storage"]

def classify_image_url(url: str):
    img, status = fetch_image(url)
    if img is None:
        return None, status
    img_emb = model.encode([img], convert_to_tensor=True)
    text_embs = model.encode(CATEGORIES, convert_to_tensor=True)
    sims = util.cos_sim(img_emb, text_embs)[0]
    idx = int(sims.argmax().item())
    return CATEGORIES[idx], f"ok_{float(sims[idx].item()):.3f}"


In [15]:
import re, json

def extract_first_url(raw):
    if pd.isna(raw) or raw is None:
        return None
    s = str(raw)
    try:
        arr = json.loads(s.replace("'", '"'))
        if isinstance(arr, list) and arr: 
            return arr[0]
    except Exception:
        pass
    m = re.findall(r"https?://[^\s,'\"]+", s)
    return m[0] if m else None

rows = df.head(10).copy()
rows["img_url"] = rows["images"].apply(extract_first_url)

out = []
for _, row in rows.iterrows():
    label, status = classify_image_url(row["img_url"])
    out.append({"title": str(row.get("title"))[:60], "predicted_category": label, "status": status})

pd.DataFrame(out)


title predicted_category  \
0  GOYMFK 1pc Free Standing Shoe Rack, Multi-laye...               None   
1  subrtex Leather ding Room, Dining Chairs Set o...               None   
2  Plant Repotting Mat MUYETOL Waterproof Transpl...               None   
3  Pickleball Doormat, Welcome Doormat Absorbent ...               None   
4  JOIN IRON Foldable TV Trays for Eating Set of ...               None   
5  LOVMOR 30'' Bathroom Vanity Sink Base Cabine, ...               None   
6  Folews Bathroom Organizer Over The Toilet Stor...               None   
7  GOYMFK 1pc Free Standing Shoe Rack, Multi-laye...               None   
8  subrtex Leather ding Room, Dining Chairs Set o...               None   
9  Plant Repotting Mat MUYETOL Waterproof Transpl...               None   

           status  
0  err_RetryError  
1  err_RetryError  
2  err_RetryError  
3  err_RetryError  
4  err_RetryError  
5  err_RetryError  
6  err_RetryError  
7  err_RetryError  
8  err_RetryError  
9  err_RetryError